In [ ]:
import numpy as np
import pandas as pd

# packages for plotting
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import urllib.request

urllib.request.urlretrieve('https://github.com/google/fonts/raw/main/ofl/ibmplexmono/IBMPlexMono-Regular.ttf', 'IBMPlexMono-Regular.ttf')
fe = font_manager.FontEntry(
    fname='IBMPlexMono-Regular.ttf',
    name='plexmono')
font_manager.fontManager.ttflist.append(fe)
plt.rcParams.update({'axes.facecolor':'#f5f4e9',
            'grid.color' : '#AAAAAA',
            'axes.edgecolor':'#333333',
            'figure.facecolor':'#FFFFFF',
            'axes.grid': False,
            'axes.prop_cycle':   plt.cycler('color', plt.cm.Dark2.colors),
            'font.family': fe.name,
            'figure.figsize': (3.5,3.5 / 1.2),
            'ytick.left': True,
            'xtick.bottom': True   ,
            'figure.dpi': 300
           })

In [ ]:
from dataclasses import dataclass

@dataclass
class GCN_Config:
    lr: float = 0.005
    hdim: int = 128
    split: float = 0.1 # 10/10/80 test val train
    # batch_size
    epochs: int = 60
    patience: int = 5
    min_delta: float = 1e-4 # for early stopping

config = GCN_Config()
seed = 511
#sensitive_threshold = 2
censor_region = 'above'
censor_split = 0.1 # 10/90 sensitive/non-sensitive data

dir_name = '../gcn_omission_results'

In [ ]:
urllib.request.urlretrieve(
    "https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/Lipophilicity.csv",
    "./lipophilicity.csv",
)
lipodata = pd.read_csv("./lipophilicity.csv")
data = list(zip(lipodata.smiles,lipodata.exp))

In [ ]:
omit_fractions = np.linspace(0, 1, int(1/0.1+1))
print(omit_fractions)

In [ ]:
import json
from datetime import datetime
today_date = datetime.today().date()
timestamp = datetime.timestamp(datetime.now())

import sys
sys.path.insert(1, '/path/to/application/app/folder')
from dglgcn import omit_train_wrapper, set_seeds

num_trials = 5

all_overall_rmse = []
all_lower_rmse = []
all_upper_rmse = []
for trial in range(num_trials):
    trial_seed = seed + trial
    sim_scores = {}
    rmse = []
    lower_rmse = []
    upper_rmse = []
    print(f'\033[46mTrial {trial+1}, Seed: {trial_seed}\033[0m')
    for i, omit_frac in enumerate(omit_fractions):
        task = f'omit_{omit_frac}'
        print(f'\n\033[34mTask: {task}\033[34m')
        set_seeds(trial_seed)
        result = omit_train_wrapper(
            data,
            censor_region,
            censor_split=censor_split,
            omit_fraction=omit_frac,
            model_config=config,
            jobname=f'trial{trial}_{task}',
            dir_name=dir_name,
            random_state=trial_seed, 
            verbose=True,
        )
        rmse.append(result[0])
        lower_rmse.append(result[1])
        upper_rmse.append(result[2])

    all_overall_rmse.append(rmse)
    all_lower_rmse.append(lower_rmse)
    all_upper_rmse.append(upper_rmse)
    
wtime_now = datetime.today().ctime()
with open(f'{dir_name}/history.json','a') as f:
    f.write(f'\nRun from today: {time_now}')
    json.dump([all_overall_rmse, all_lower_rmse, all_upper_rmse],f)

In [ ]:
df = pd.DataFrame({
    'omit frac': omit_fractions,
    'overall rmse': np.mean(all_overall_rmse, axis=0),
    'overall rmse std': np.std(all_overall_rmse, axis=0),
    'lower rmse': np.mean(all_lower_rmse, axis=0),
    'lower rmse std': np.std(all_lower_rmse, axis=0),
    'upper rmse': np.mean(all_upper_rmse, axis=0),
    'upper rmse std': np.std(all_upper_rmse, axis=0),
})

df.to_json(f'{dir_name}/summary_{today_date}.json')
df

In [ ]:
plt.figure(figsize=(9,9))

if sensitive_threshold=='above':
    lower_rmse_label = 'Non-sensitive Region' # s=1
    upper_rmse_label = 'Sensitive Region' # s=0
else:
    lower_rmse_label = 'Sensitive Region' 
    upper_rmse_label = 'Non-sensitive Region'    

plt.errorbar(df['omit frac'], df['lower rmse'], yerr=df['lower rmse std'], capsize=5, fmt='-o', label=lower_rmse_label)
plt.errorbar(df['omit frac'], df['upper rmse'], yerr=df['upper rmse std'], capsize=5, fmt='-o', label=upper_rmse_label)

plt.title('Test Errors')
plt.xlabel('% Sensitive Data Omitted')
plt.ylabel('RMSE')
#plt.ylim(0,1.6)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(f'{dir_name}/gcn_testerror_omit_{today_date}.png', dpi=300)
plt.show()

In [ ]:
if sensitive_threshold=='above':
    lower_rmse_label = 'Non-sensitive Region' # s=0
    upper_rmse_label = 'Sensitive Region' # s=1
else:
    lower_rmse_label = 'Sensitive Region' 
    upper_rmse_label = 'Non-sensitive Region'   

plt.plot(df['omit frac'], df['lower rmse'], label=lower_rmse_label, marker='o', markersize=2)
plt.fill_between(df['omit frac'], df['lower rmse'] - df['lower rmse std'], df['lower rmse'] + df['lower rmse std'], alpha=0.2)

plt.plot(df['omit frac'], df['upper rmse'], label=upper_rmse_label, marker='o', markersize=2)
plt.fill_between(df['omit frac'], df['upper rmse'] - df['upper rmse std'], df['upper rmse'] + df['upper rmse std'], alpha=0.2)

plt.title('Test Errors')
plt.xlabel('Similarity Score Intervals')
plt.ylabel('RMSE')
plt.ylim(0,1.6)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(f'{dir_name}/gcn_testerror_omit_{today_date}_v2.png', dpi=300)
plt.show()